<a href="https://colab.research.google.com/github/weedge/doraemon-nb/blob/main/achatbot_moshi_bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# init

In [ ]:
!cd /content && rm -rf achatbot && git clone --recursive https://github.com/ai-bot-pro/achatbot.git

In [14]:
%cd /content/achatbot


/content/achatbot


In [ ]:
!bash scripts/pypi_achatbot.sh dev


In [ ]:
!pip install -q "dist/achatbot-0.0.7.2-py3-none-any.whl[fastapi_bot_server,ngrok_proxy,websocket_server_transport,daily_room_audio_stream,livekit_room_audio_stream,silero_vad_analyzer,moshi_voice_processor,queue]"


# run livekit webrtc room bot

In [15]:
!cat /content/livekit_moshi_voice_bot.json

{
  "chat_bot_name": "LivekitMoshiVoiceBot",
  "room_name": "chat-room",
  "room_url": "",
  "token": "",
  "room_manager": {
    "tag": "livekit_room",
    "args": {
      "bot_name": "LivekitMoshiVoiceBot",
      "is_common_session": false
    }
  },
  "services": {
    "pipeline": "achatbot",
    "vad": "silero",
    "voice_llm": "moshi"
  },
  "config": {
    "vad": {
      "tag": "silero_vad_analyzer",
      "args": { "stop_secs": 0.7 }
    },
    "voice_llm": {
      "tag": "moshi_voice_processor",
      "args": {
        "model_name": "kyutai/moshiko-pytorch-bf16",
        "lm_gen_args": {
          "use_sampling": true,
          "temp": 0.8,
          "temp_text": 0.7,
          "top_k": 250,
          "top_k_text": 25,
          "check": false
        }
      }
    }
  },
  "config_list": []
}


In [6]:
from google.colab import userdata
HF_TOKEN=userdata.get('HF_TOKEN')
DAILY_API_KEY=userdata.get('DAILY_API_KEY')

LIVEKIT_URL=userdata.get('LIVEKIT_URL')
LIVEKIT_API_KEY=userdata.get('LIVEKIT_API_KEY')
LIVEKIT_API_SECRET=userdata.get('LIVEKIT_API_SECRET')



In [ ]:
!LIVEKIT_URL=$LIVEKIT_URL LIVEKIT_API_KEY=$LIVEKIT_API_KEY LIVEKIT_API_SECRET=$LIVEKIT_API_SECRET \
  python -m src.cmd.bots.main -f /content/livekit_moshi_voice_bot.json

# run fastapi websocket bot

In [5]:
from google.colab import userdata
NGROK_TOKEN=userdata.get('NGROK_TOKEN')

In [6]:
!ngrok config add-authtoken $NGROK_TOKEN

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [7]:
!cat /content/fastapi_websocket_moshi_voice_bot.json

{
  "chat_bot_name": "FastapiWebsocketMoshiVoiceBot",
  "transport_type": "websocket",
  "handle_sigint": false,
  "services": {
    "pipeline": "achatbot",
    "vad": "silero",
    "voice_llm": "moshi"
  },
  "config": {
    "vad": {
      "tag": "silero_vad_analyzer",
      "args": { "stop_secs": 0.7 }
    },
    "voice_llm": {
      "tag": "moshi_opus_voice_processor",
      "args": {
        "model_name": "kyutai/moshiko-pytorch-bf16",
        "lm_gen_args": {
          "use_sampling": true,
          "temp": 0.8,
          "temp_text": 0.7,
          "top_k": 250,
          "top_k_text": 25,
          "check": false
        }
      }
    }
  },
  "config_list": []
}


In [ ]:
!python -m src.cmd.websocket.server.fastapi_ws_bot_serve --port 8000 --ngrok -f /content/fastapi_websocket_moshi_voice_bot.json